# Segmenting and Clustering Neighborhoods in Toronto
In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

In [7]:
# import libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

print("Libraries imported")

Libraries imported


**Scrape the wikipedia page for Toronto neighborhood data and wrangle into a dataframe**

In [15]:
# grab the data from wikipediea
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(wiki_url).text
canada_xml = bs(source, 'lxml')
table=canada_xml.find('table')


In [49]:
# grab the columns Postalcode,Borough and Neighborhood
column_names = ['Postal Code','Borough','Neighborhood']
df_canada = pd.DataFrame(columns = column_names)
df_canada.head()

,Postal Code,Borough,Neighborhood


In [126]:
# build the table
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df_canada.loc[len(df_canada)] = row_data
df_canada.head()        

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


**Let's cleanup some data**

In [95]:
# Get rid of the not assigned boroughs; group them by postalcodes and boroughs
df_boroughs = df_canada[df_canada.Borough != "Not assigned"].reset_index()
df_boroughs= df_boroughs.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
# Are there any neighborhoods not assigned?
(df_boroughs['Neighborhood'] == "Not assigned").any()
df_boroughs.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge, Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,Scarborough,"Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae"


**We don't have any "Not Assigned" neighborhoods since the answer to the last answer is False.**

In [43]:
df_boroughs.shape

(103, 3)

**We have 103 rows and three columns.**

## Part 2: Working with the latitude and longitude

In [44]:
# get the data and create the dataframe
geo_url='https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
df_geo=pd.read_csv(geo_url)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [53]:
df_hoods = pd.merge(df_boroughs, df_geo, on = 'Postal Code')
df_hoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Part 3: Exploring and mapping

In [55]:
# need more libraries
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                                                                                 -failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - cffi -> python[version='2.7.*|3.5.*|3.6.*|3.6.12|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.9,<3.10.0a0|>=3.8,<3.9.0a0|3.6.9|3.6.9|3.6.9|>=2.7,<2.8.0a0|3.6.9|>=3.5,<3.6.0a0|3.4.*',build='0_73_pypy|4_73_pypy|3_73_pypy|2_73_pypy|1_73_pypy']
  - geopy -> python[version='2.7.*|3.4.*|3.5.*|

ModuleNotFoundError: No module named 'folium'

In [57]:
!pip install folium


     |████████████████████████████████| 94 kB 6.3 MB/s  eta 0:00:01


In [63]:
import folium

In [64]:
#get coordinates of Toronto
address = 'Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))



The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [68]:
# Create a map of the city
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

map_toronto

**Maps of the neighborhoods**

In [70]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
# add markers to map
for lat, lng, borough, neighborhood in zip(df_hoods['Latitude'], df_hoods['Longitude'], df_hoods['Borough'], df_hoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [119]:
# filter borough names that contain the word downtown
borough_names = list(df_canada.Borough.unique())

boros_with_downtown = []

for x in borough_names:
    if "downtown" in x.lower():
        boros_with_downtown.append(x)
        
print(boros_with_downtown)

# create a new DataFrame with only boroughs that contain the word downtown
df_downtown = df_canada[df_canada['Borough'].isin(boros_with_downtown)].reset_index(drop=True)

# merge with geo data
df_downtown = pd.merge(df_downtown, df_geo, on = 'Postal Code')
df_downtown.head()

['Downtown Toronto']


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
2,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
3,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [135]:
downtown_data = df_canada[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

['Regent Park, Harbourfront', "Queen's Park, Ontario Provincial Government", 'Garden District, Ryerson', 'St. James Town', 'Berczy Park', 'Central Bay Street', 'Christie', 'Richmond, Adelaide, King', 'Harbourfront East, Union Station, Toronto Islands', 'Toronto Dominion Centre, Design Exchange', 'Commerce Court, Victoria Hotel', 'University of Toronto, Harbord', 'Kensington Market, Chinatown, Grange Park', 'CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport', 'Rosedale', 'Stn A PO Boxes', 'St. James Town, Cabbagetown', 'First Canadian Place, Underground city', 'Church and Wellesley']


In [101]:
map_downtown_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
# add markers to map
for lat, lng, borough, neighborhood in zip(df_downtown['Latitude'], df_downtown['Longitude'], df_downtown['Borough'], df_downtown['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_downtown_toronto)  
    
map_downtown_toronto

## Let's play FourSquare

In [82]:
#authorizations
CLIENT_ID = 'EP1J3Q1TF1SXGKBJH41IVHUZB0BQUIYA4BKPY302OOLW1Z5T' # your Foursquare ID
CLIENT_SECRET = 'M4GTHI2HIQNITWFJLVHKV0KQGN50WUGQWUOUUZH3PE1OIOBQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value


**What are the venues downtown?**

In [83]:
# function to scroll through nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [138]:
#need data for downtown venues
downtown_venues = getNearbyVenues(names=df_downtown['Neighborhood'],
                                   latitudes=df_downtown['Latitude'],
                                   longitudes=df_downtown['Longitude']
                                  )
print(downtown_venues.shape)

Regent Park, Harbourfront
Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
Garden District, Ryerson
St. James Town
St. James Town
Berczy Park
Berczy Park
Central Bay Street
Central Bay Street
Christie
Christie
Richmond, Adelaide, King
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Commerce Court, Victoria Hotel
University of Toronto, Harbord
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Rosedale
Stn A PO Boxes
Stn A PO Boxe

In [103]:
downtown_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [139]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))


There are 207 uniques categories.


## Analyze neighborhoods

In [140]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [141]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.016949,0.000000,0.000000,0.00,0.00,0.016949,0.00000,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
2,Central Bay Street,0.016393,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.032787,0.000000,0.000000,0.00,0.00,0.016393,0.00000,0.000000,0.016393,0.000000
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,...,0.012821,0.012821,0.012821,0.00,0.00,0.000000,0.00000,0.012821,0.000000,0.000000
5,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.020000,0.000000,0.000000,0.00,0.00,0.020000,0.00000,0.000000,0.010000,0.000000
6,"First Canadian Place, Underground city",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.020000,0.010000,0.000000,0.00,0.01,0.010000,0.00000,0.000000,0.010000,0.000000
7,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.020000,0.000000,0.00,0.00,0.000000,0.01000,0.010000,0.010000,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.010000,0.000000,0.00,0.01,0.010000,0.00000,0.000000,0.010000,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.045455,0.00000,0.045455,0.015152,0.000000


In [143]:
#put into data frame 
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Restaurant,Farmers Market,Seafood Restaurant,Beer Bar,Bakery,Breakfast Spot,Irish Pub
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Plane,Harbor / Marina,Sculpture Garden,Boat or Ferry,Rental Car Location,Coffee Shop,Boutique
2,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Bubble Tea Shop,Burger Joint,Salad Place,Japanese Restaurant,Thai Restaurant,Middle Eastern Restaurant
3,Christie,Grocery Store,Café,Park,Candy Store,Restaurant,Italian Restaurant,Baby Store,Athletics & Sports,Coffee Shop,Nightclub
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Fast Food Restaurant,Yoga Studio,Pub,Café,Hotel
5,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Hotel,Café,Gym,American Restaurant,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Deli / Bodega
6,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Gym,American Restaurant,Steakhouse,Deli / Bodega,Asian Restaurant
7,"Garden District, Ryerson",Clothing Store,Coffee Shop,Café,Japanese Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Hotel,Cosmetics Shop,Lingerie Store,Italian Restaurant
8,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Hotel,Café,Brewery,Restaurant,Fried Chicken Joint,Italian Restaurant,Scenic Lookout,History Museum
9,"Kensington Market, Chinatown, Grange Park",Café,Coffee Shop,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Dessert Shop,Bar,Farmers Market,Bakery,Arts & Crafts Store


Surprise! Coffee shops are popular downtown

## Cluster the neighborhoods

In [145]:
# new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 3, 0, 4, 2, 2, 2, 2, 2, 2], dtype=int32)

In [148]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = df_downtown

# merge downtown_grouped with downtown_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() 

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Café,Pub,Breakfast Spot,Theater,Wine Shop,Electronics Store,Performing Arts Venue
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Café,Pub,Breakfast Spot,Theater,Wine Shop,Electronics Store,Performing Arts Venue
2,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Discount Store,Bar,Smoothie Shop,Italian Restaurant,Japanese Restaurant,Beer Bar,Sandwich Place
3,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Discount Store,Bar,Smoothie Shop,Italian Restaurant,Japanese Restaurant,Beer Bar,Sandwich Place
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Clothing Store,Coffee Shop,Café,Japanese Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Hotel,Cosmetics Shop,Lingerie Store,Italian Restaurant


In [151]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters